<a href="https://colab.research.google.com/github/szwright98/ZeroAvia/blob/master/Model1MissionProfileforFuelszw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
class mp_phase:
    def __init__(self, num = 0, powfrac = 0, dur = 0, min_dur = 0, max_dur = 0, ref_alt = 0, alt_start = 0, alt_end = 0, cool_up = 0, cool_low = 0):
        self.phase = num # units: none, numerical designation of flight phase (1 = start-up, 2 = taxi, etc.) 
        self.propulsion_system_pwr = powfrac  # units: 
        self.duration = dur # units:
        self.duration_min = min_dur
        self.duration_max = max_dur
        self.ref_altitude = ref_alt #units 
        self.altitude_start = 0 # units: 
        self.altitude_end = 0 # units: 
        self.coolant_upper = 0 # units: 
        self.coolant_lower = 0 # units:
    def phase_powertime(self, motor_eff = .98):
        return self.duration * self.propulsion_system_pwr * (1./(motor_eff))
    def phase_energy(self, max_motor_power, motor_eff = .98):
        return self.duration * self.propulsion_system_pwr * max_motor_power*(1./(motor_eff))
"""
class mission_profile:

    def __init__(self):
    class start_up_phase(mp_phase):
        def __init__(self):
            super().__init__()
            self.phase = 1
    class taxi_phase(mp_phase):
        def __init__(self):
            super().__init__()
            self.phase = 2
    class take_off_phase(mp_phase):
        def __init__(self):
            super().__init__()
            self.phase = 3
    class climb_phase(mp_phase):
        def __init__(self):
            super().__init__()
            self.phase = 4
    class cruise_phase(mp_phase):
        def __init__(self):
            super().__init__()
            self.phase = 5
    class descent_phase(mp_phase):
        def __init__(self):
            super().__init__()
            self.phase = 6
    class go_around_phase(mp_phase):
        def __init__(self):
            super().__init__()
            self.phase = 7
    class land_phase(mp_phase):
        def __init__(self):
            super().__init__()
            self.phase = 8
            """

'\nclass mission_profile:\n\n    def __init__(self):\n    class start_up_phase(mp_phase):\n        def __init__(self):\n            super().__init__()\n            self.phase = 1\n    class taxi_phase(mp_phase):\n        def __init__(self):\n            super().__init__()\n            self.phase = 2\n    class take_off_phase(mp_phase):\n        def __init__(self):\n            super().__init__()\n            self.phase = 3\n    class climb_phase(mp_phase):\n        def __init__(self):\n            super().__init__()\n            self.phase = 4\n    class cruise_phase(mp_phase):\n        def __init__(self):\n            super().__init__()\n            self.phase = 5\n    class descent_phase(mp_phase):\n        def __init__(self):\n            super().__init__()\n            self.phase = 6\n    class go_around_phase(mp_phase):\n        def __init__(self):\n            super().__init__()\n            self.phase = 7\n    class land_phase(mp_phase):\n        def __init__(self):\n           

In [55]:
#test_mp = mission_profile()

### MAKE THIS A FUNCTION TO TAKE INPUTS AND ORDER THEM INTO A LIST OF MISSION PROFILE OBJECTS
def get_MissionProfileList(phase_nums,power_fractions,ref_durations):
  ph = []
  for x in range(len(phase_nums)):
    #print(len(ph))
    phase_obj = mp_phase(phase_nums[x],power_fractions[x],ref_durations[x])
    ph.append(phase_obj)
  return ph 
  #print('done')
#print(ph)

### MAKE THIS A FUNCTION THAT TAKES IN MAX MOTOR POWER AND GIVES TOTAL MOTORSIDE (I.E. WITHOUT ANY ENERGY LOSSES FROM THE POWERTRAIN) ENERGY PER PHASE FOR THE FLIGHT MISSION PROFILE
def get_MotorsideEnergy(phaselist, max_power):
  tot_en = [] 
  for x in range(len(phaselist)):
    tot_en.append(phaselist[x].phase_energy(max_power))
  return tot_en

#print(tot_en)
#ph[0].propulsion_system_pwr = .1
#print(ph[0].propulsion_system_pwr)
#test_cr = test_mp.cruise_phase()
#test_cr.duration = 1.
#test_cr.propulsion_system_pwr = .5
#print(test_cr.duration)
#a = test_cr.phase_powertime()
#c = test_cr.phase_energy(600, motor_eff = 0.5)
#b = test_cr.phase_energy(600)
#print(a)
#print(b)
#print(c)
#print(test_cr.abb)

In [56]:
#import test


tank_volume = 10 
inter_qty = 3
max_power = 600

class fuel_tank:
    def __init__(self,tank_volume = 0):
        self.max = tank_volume
        self.min = 0
        self.specific_energy = 39.4 #kWhr/kg
        self.energy_density = 1.479 #kWhr/L

    def specific_energy(self, specific_energy):
        # return specific energy of tank volume
        # base value is in kWhr/kg
        return specific_energy
    
    def fuel_vol_needed(self, en_needed):
        # calculate fuel volume needed as a function of the energy needed and energy density
        # takes en_needed in kWhr
        # energy density is in kWhr/L
        # returns fuel mass in liters
        return en_needed/self.energy_density

    def fuel_mass_needed (self, en_needed):
        # calculate fuel mass needed as a function of the energy needed (mission phase or overall) and the specific energy. 
        # takes needed in kWhr
        # specific energy is in kWhr/kg 
        # returns fuel mass in kg
        return en_needed/self.specific_energy 

    def __repr__(self):
        return "fuel_tank('{}')".format(self.tank_volume)

    def __str__(self):
        return "fuel tank volume: {}".format(self.tank_volume)
        
class fuel_cell:
     def __init__(self):
         self.efficiency = .5
     def get_losses(self,phase_info):
        losses = []
        for x in range(len(phase_info)):
          losses.append(phase_info[x]*(1-self.efficiency))
        return losses


class Pow_Dis_Unt:
    def __init__(self, AWG = 0, material = '', FC_cable_length = 0, PDU_cable_length = 0, contactor_resistance = 0):
        self.AWG = AWG
        self.material = material
        self.FC_cable_length = FC_cable_length
        self.PDU_cable_length = PDU_cable_length
        self.contactor_resistance = contactor_resistance
        self.efficiency = 1.
    def get_losses(self,phase_info):
        losses = []
        for x in range(len(phase_info)):
          losses.append(phase_info[x]*(1-self.efficiency))
        return losses

class inverter:
    def __init__(self, max_power = 0, mass = 0, volume = 0, efficiency = .98):
        self.max_power = max_power
        self.mass = mass
        self.volume = volume
        self.efficiency = efficiency
        self.num_of_inverters = 0
        self.chain_efficiency = efficiency**self.num_of_inverters
    def get_losses(self,phase_info):
        losses = []
        for x in range(len(phase_info)):
          #print(phase_info)
          losses.append(phase_info[x]*(1-self.efficiency))
        return losses

class motor:
    def __init__(self, max_power = 0, max_torque = 0,weight = 0, volume = 0, nominal_speed = 0,efficiency = .97):
        self.max_power = max_power
        self.max_torque = max_torque
        self.weight = weight
        self.volume = volume
        self.max_nominal_speed = nominal_speed
        self.efficiency = efficiency
    def get_losses(self,phase_info):
        losses = []
        for x in range(len(phase_info)):
          losses.append(phase_info[x]*(1-self.efficiency))
        return losses

In [57]:
## create the objects 


## inputs from mission flight profile go here.
max_power1 = 600
power_fractions1 = [.05,.15,1,.8,.5,.2,.8,.15]
ref_durations1 = [.25,.167,.0833,.333,1,.167,.167,.167]
phase_nums1 = [1, 2, 3, 4, 5, 6, 7, 8]

mprof1 = get_MissionProfileList(phase_nums1,power_fractions1,ref_durations1)



fueltank_in = fuel_tank()
fuelcell_in = fuel_cell()
PowDisUnit_in = Pow_Dis_Unt()
inv_in = inverter()
motor_in = motor(max_power = max_power1)


tot_en1 = get_MotorsideEnergy(mprof1, motor_in.max_power)



In [58]:
"""
import numpy as np
index = np.where(mprof1.phase == 1)

print(index)
"""

'\nimport numpy as np\nindex = np.where(mprof1.phase == 1)\n\nprint(index)\n'

In [59]:
## define more functions 

## GET THE MOTOR ENERGY/POWER LOSSES PER PHASE MAKE THIS A METHOD FOR MOTOR CLASS
def get_motor_losses(phase_info,motor_eta):
  return

def get_inverter_losses(phase_info,inv_eta):
  inv_ens = []
  for x in range(len(phase_info)):
    inv_ens.append(phase_info[x]*(1-inv_eta))
  return inv_ens

def get_fcell_losses(phase_info,fcell_eta):
  fcell_ens = []
  for x in range(len(phase_info)):
    fcell_ens.append(phase_info[x]*(1-fcell_eta))
  return fcell_ens

In [60]:
def get_PowerTrain_Energy(motor,fuelcell,inverter,phase_energy):
  a = motor.get_losses(phase_energy)
  b = inverter.get_losses(phase_energy)
  c = fuelcell.get_losses(phase_energy)
  fuel_energy = [o + a + b + c for o,a,b,c in zip(phase_energy,a,b,c)]
  return fuel_energy
fuel_energy = get_PowerTrain_Energy(motor_in, fuelcell_in, inv_in, tot_en1)

In [61]:
import numpy as np

print(fueltank_in.fuel_mass_needed(np.sum(fuel_energy)))
print(fueltank_in.fuel_vol_needed(np.sum(fuel_energy)))


25.995778514451477
692.5176967338662
